In [ ]:
import pandas as pd
import numpy as np
from keras import layers
import tensorflow as tf
import keras
from tensorflow import keras
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load data for training GAN

In [ ]:
wafer_map = pd.read_pickle('/content/drive/MyDrive/Capstone_programming/Data_final/wafer_map_for_GAN.pkl')
wafer_label = pd.read_pickle('/content/drive/MyDrive/Capstone_programming/Data_final/wafer_label_for_GAN.pkl')

## Median Filter

In [ ]:
#median function
from skimage.filters.rank import median
from skimage.morphology import disk
def median_img(x):
  img_out = median(x,disk(1.5))
  return img_out


In [ ]:
median_image = []
median_label = []

for i in range(len(x)): 
      median_image.append(median_img(x[i]))
      median_label.append(y[i])
    
median_image = np.array(median_image)
median_label = np.array(median_label)    

## cGAN model

### Set up

In [ ]:
batch_size = 32
num_channels = 1
num_classes = 8
image_size = 28
latent_dim = 128

In [ ]:
generator_in_channels = latent_dim + num_classes
discriminator_in_channels = num_channels + num_classes
print(generator_in_channels, discriminator_in_channels)

136 9


In [ ]:
gan_wafer = median_image
gan_labels = median_label

# Scale the pixel values to [0, 1] range, add a channel dimension to
# the images, and one-hot encode the labels.
gan_wafer = gan_wafer.astype("float32") 
gan_wafer *= 127.5
gan_wafer = gan_wafer/255
gan_wafer = np.reshape(gan_wafer, (-1, 28, 28, 1))
gan_labels = keras.utils.to_categorical(gan_labels, num_classes )

# Create tf.data.Dataset.
dataset = tf.data.Dataset.from_tensor_slices((gan_wafer, gan_labels))
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

print(f"Shape of training images: {gan_wafer.shape}")
print(f"Shape of training labels: {gan_labels.shape}")

Shape of training images: (3675, 28, 28, 1)
Shape of training labels: (3675, 8)


### GAN architecture

In [ ]:
discriminator = keras.Sequential(
    [
        keras.layers.InputLayer((28, 28, discriminator_in_channels)),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(),
        layers.GlobalMaxPooling2D(),
        layers.Dense(1,activation="sigmoid"),
    ],
    name="discriminator",
)

# Create the generator.
generator = keras.Sequential(
    [
        keras.layers.InputLayer((generator_in_channels,)),
        # We want to generate 128 + num_classes coefficients to reshape into a
        # 7x7x(128 + num_classes) map.
        layers.Dense(7 * 7 * generator_in_channels),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((7, 7, generator_in_channels)),
        layers.Conv2DTranspose(256, (4, 4),strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(),
        layers.Conv2D(1, (7, 7), padding="same", activation="tanh"),
    ],
    name="generator",
)


## GAN training

In [ ]:
class ConditionalGAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super(ConditionalGAN, self).__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.gen_loss_tracker = keras.metrics.Mean(name="generator_loss")
        self.disc_loss_tracker = keras.metrics.Mean(name="discriminator_loss")

    @property
    def metrics(self):
        return [self.gen_loss_tracker, self.disc_loss_tracker]

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(ConditionalGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, data):
        # Unpack the data.
        real_images, one_hot_labels = data

        # Add dummy dimensions to the labels so that they can be concatenated with
        # the images. This is for the discriminator.
        image_one_hot_labels = one_hot_labels[:, :, None, None]
        image_one_hot_labels = tf.repeat(
            image_one_hot_labels, repeats=[image_size * image_size]
        )
        image_one_hot_labels = tf.reshape(
            image_one_hot_labels, (-1, image_size, image_size, num_classes)
        )

        # Sample random points in the latent space and concatenate the labels.
        # This is for the generator.
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # Decode the noise (guided by labels) to fake images.
        generated_images = self.generator(random_vector_labels)

        # Combine them with real images. Note that we are concatenating the labels
        # with these images here.
        fake_image_and_labels = tf.concat([generated_images, image_one_hot_labels], -1)
        real_image_and_labels = tf.concat([real_images, image_one_hot_labels], -1)
        combined_images = tf.concat(
            [fake_image_and_labels, real_image_and_labels], axis=0
        )

        # Assemble labels discriminating real from fake images.
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )

        # Train the discriminator.
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space.
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # Assemble labels that say "all real images".
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            fake_images = self.generator(random_vector_labels)
            fake_image_and_labels = tf.concat([fake_images, image_one_hot_labels], -1)
            predictions = self.discriminator(fake_image_and_labels)
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Monitor loss.
        self.gen_loss_tracker.update_state(g_loss)
        self.disc_loss_tracker.update_state(d_loss)
        return {
            "g_loss": self.gen_loss_tracker.result(),
            "d_loss": self.disc_loss_tracker.result(),
        }

In [ ]:
cond_gan = ConditionalGAN(
    discriminator=discriminator, generator=generator, latent_dim=latent_dim
)
cond_gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0002,beta_1=0.5),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0002,beta_1=0.5),
    loss_fn=keras.losses.BinaryCrossentropy(from_logits=False),
)

cond_gan.fit(dataset, epochs=150)

Epoch 1/150
115/115 [==============================] - 5s 19ms/step - g_loss: 0.8192 - d_loss: 0.6372
Epoch 2/150
115/115 [==============================] - 2s 18ms/step - g_loss: 0.8803 - d_loss: 0.5774
Epoch 3/150
115/115 [==============================] - 2s 18ms/step - g_loss: 0.7633 - d_loss: 0.6432
Epoch 4/150
115/115 [==============================] - 2s 18ms/step - g_loss: 0.7738 - d_loss: 0.6418
Epoch 5/150
115/115 [==============================] - 2s 18ms/step - g_loss: 0.8062 - d_loss: 0.6286
Epoch 6/150
115/115 [==============================] - 2s 18ms/step - g_loss: 0.8223 - d_loss: 0.6281
Epoch 7/150
115/115 [==============================] - 2s 18ms/step - g_loss: 0.8155 - d_loss: 0.6331
Epoch 8/150
115/115 [==============================] - 2s 19ms/step - g_loss: 0.8094 - d_loss: 0.6185
Epoch 9/150
115/115 [==============================] - 2s 20ms/step - g_loss: 0.8341 - d_loss: 0.6357
Epoch 10/150
115/115 [==============================] - 2s 20ms/step - g_loss: 0.8

## Using GAN to balance

In [ ]:
def interpolate_image(num_label,num_interpolation,gan_generator):
    # Sample noise for the interpolation.
    interpolation_noise = tf.random.normal(shape=(1, latent_dim))
    interpolation_noise = tf.repeat(interpolation_noise, repeats=num_interpolation)
    interpolation_noise = tf.reshape(interpolation_noise, (num_interpolation, latent_dim))
    # Convert the start and end labels to one-hot encoded vectors.
    label = keras.utils.to_categorical([num_label], num_classes)
    label = tf.cast(label, tf.float32)

    # Calculate the interpolation vector between the two labels.
    percent_label = tf.linspace(0, 1, num_interpolation)[:, None]
    percent_label = tf.cast(percent_label, tf.float32)
    interpolation_labels = (
        label * (1 - percent_label +  percent_label )
        )

    # Combine the noise and the labels and run inference with the generator.
    noise_and_labels = tf.concat([interpolation_noise,  interpolation_labels], 1)
    fake_image = gan_generator.predict(noise_and_labels)
    return fake_image,  interpolation_labels 